In [1]:
from simtk import openmm
from simtk.openmm import app, unit

import time
import random
import numpy as np

In [2]:
def get_2_particle_system_and_positions(expression):
    
    # Create system and add particles
    system = openmm.System()
    system.addParticle(22.99)
    system.addParticle(35.45)
    system.setDefaultPeriodicBoxVectors(openmm.Vec3(9, 0, 0), openmm.Vec3(0, 9, 0), openmm.Vec3(0, 0, 9))
    
    # Create CustomNonbondedForce (will add particles to the force later)
    nb_force = openmm.CustomNonbondedForce(expression)
    system.addForce(nb_force)
    nb_force.setNonbondedMethod(openmm.CustomNonbondedForce.CutoffPeriodic)
    nb_force.setCutoffDistance(0.1 * unit.nanometer)
    
    # Create positions
    positions = [openmm.Vec3(random.uniform(0, 9)*0.1, 0, 0) for _ in range(2)]
    
    return system, positions


In [3]:
def run_experiment(system, positions):
    # Set up context
    integrator = openmm.LangevinMiddleIntegrator(300, 1.0, 0.004)
    platform = openmm.Platform.getPlatformByName("CUDA")
    context = openmm.Context(system, integrator, platform)
    context.setPositions(positions)

    # Time getState()
    force_group_idx = 0
    times = []
    for i in range(5):
        context.setParameter('lambda', 1 - i*0.1)
        initial_time = time.time()
        context.getState(getEnergy=True, groups=2**force_group_idx)
        elapsed_time = (time.time() - initial_time)
        print(elapsed_time)
        times.append(elapsed_time)
    print("mean: ", np.mean(times[1:]))

# With simple expression and no per particle params, long range correction off

In [4]:
simple_expression = "U_sterics; U_sterics = lambda * 4 * epsilon * x * (x - 1.0); x = (sigma / r_eff_sterics)^6; sigma = 1; epsilon = 1; r_eff_sterics = sqrt(r^2 + w_sterics^2); w_sterics = 0.3;"


In [5]:
# Create system and customize it for this experiment
system, positions = get_2_particle_system_and_positions(simple_expression)
nb_force = system.getForce(0)
nb_force.setUseLongRangeCorrection(False)
nb_force.addGlobalParameter('lambda', 1.0)
nb_force.addParticle([])
nb_force.addParticle([])

# Run experiment
run_experiment(system, positions)

0.0013477802276611328
0.00010633468627929688
9.799003601074219e-05
8.940696716308594e-05
8.630752563476562e-05
mean:  9.500980377197266e-05


# With simple expression and no per particle params, long range correction on

In [4]:
simple_expression = "U_sterics; U_sterics = lambda * 4 * epsilon * x * (x - 1.0); x = (sigma / r_eff_sterics)^6; sigma = 1; epsilon = 1; r_eff_sterics = sqrt(r^2 + w_sterics^2); w_sterics = 0.3;"


In [5]:
# Create system and customize it for this experiment
system, positions = get_2_particle_system_and_positions(simple_expression)
nb_force = system.getForce(0)
nb_force.setUseLongRangeCorrection(True)
nb_force.addGlobalParameter('lambda', 1.0)
nb_force.addParticle([])
nb_force.addParticle([])

# Run experiment
run_experiment(system, positions)

0.005000591278076172
0.0017299652099609375
0.0016379356384277344
0.001554727554321289
0.0016150474548339844
mean:  0.0016344189643859863


# With simple expression and many per particle params, long range correction off

In [4]:
simple_expression = "U_sterics; U_sterics = lambda * 4 * epsilon * x * (x - 1.0); x = (sigma / r_eff_sterics)^6; sigma = 1; epsilon = 1; r_eff_sterics = sqrt(r^2 + w_sterics^2); w_sterics = 0.3;"
# simple_expression = "U_sterics; U_sterics = lambda * 4 * epsilon * x * (x - 1.0); x = (sigma / r_eff_sterics)^6; sigma = sigma_old1; epsilon = epsilon_old1 * is_rest * (is_unique_old + is_core + is_unique_new); r_eff_sterics = sqrt(r^2 + w_sterics^2); w_sterics = 0.3;"


In [5]:
# Create system and customize it for this experiment
system, positions = get_2_particle_system_and_positions(simple_expression)
nb_force = system.getForce(0)
nb_force.setUseLongRangeCorrection(False)
nb_force.addGlobalParameter('lambda', 1.0)

# Add per-particle parameters for rest scaling -- these three sets are disjoint
nb_force.addPerParticleParameter("is_rest")
nb_force.addPerParticleParameter("is_nonrest_solute")
nb_force.addPerParticleParameter("is_nonrest_solvent")

# Add per-particle parameters for alchemical scaling -- these sets are also disjoint
nb_force.addPerParticleParameter('is_environment')
nb_force.addPerParticleParameter('is_core')
nb_force.addPerParticleParameter('is_unique_old')
nb_force.addPerParticleParameter('is_unique_new')

# Add per-particle parameters for defining energy
nb_force.addPerParticleParameter("sigma_old")
nb_force.addPerParticleParameter("sigma_new")
nb_force.addPerParticleParameter("epsilon_old")
nb_force.addPerParticleParameter("epsilon_new")

nb_force.addParticle([1, 0, 0, 0, 0, 1, 0, 0.24, 0.24, 0.37, 0.37])
nb_force.addParticle([1, 0, 0, 0, 1, 0, 0, 0.45, 0.45, 0.15, 0.15])


# Run experiment
run_experiment(system, positions)

0.001435995101928711
0.00011420249938964844
9.5367431640625e-05
9.560585021972656e-05
9.608268737792969e-05
mean:  0.00010031461715698242


# With simple expression and many per particle params, long range correction on

In [4]:
simple_expression = "U_sterics; U_sterics = lambda * 4 * epsilon * x * (x - 1.0); x = (sigma / r_eff_sterics)^6; sigma = 1; epsilon = 1; r_eff_sterics = sqrt(r^2 + w_sterics^2); w_sterics = 0.3;"


In [5]:
# Create system and customize it for this experiment
system, positions = get_2_particle_system_and_positions(simple_expression)
nb_force = system.getForce(0)
nb_force.setUseLongRangeCorrection(True)
nb_force.addGlobalParameter('lambda', 1.0)

# Add per-particle parameters for rest scaling -- these three sets are disjoint
nb_force.addPerParticleParameter("is_rest")
nb_force.addPerParticleParameter("is_nonrest_solute")
nb_force.addPerParticleParameter("is_nonrest_solvent")

# Add per-particle parameters for alchemical scaling -- these sets are also disjoint
nb_force.addPerParticleParameter('is_environment')
nb_force.addPerParticleParameter('is_core')
nb_force.addPerParticleParameter('is_unique_old')
nb_force.addPerParticleParameter('is_unique_new')

# Add per-particle parameters for defining energy
nb_force.addPerParticleParameter("sigma_old")
nb_force.addPerParticleParameter("sigma_new")
nb_force.addPerParticleParameter("epsilon_old")
nb_force.addPerParticleParameter("epsilon_new")

nb_force.addParticle([1, 0, 0, 0, 0, 1, 0, 0.24, 0.24, 0.37, 0.37])
nb_force.addParticle([1, 0, 0, 0, 1, 0, 0, 0.45, 0.45, 0.15, 0.15])


# Run experiment
run_experiment(system, positions)

0.0053102970123291016
0.0018839836120605469
0.001558065414428711
0.0017595291137695312
0.0015594959259033203
mean:  0.0016902685165405273


# With complex expression and many per particle params, long range correction off

In [4]:
_default_sterics_expression_list = [

        "U_sterics;",

        # Define sterics functional form
        "U_sterics = 4 * epsilon * x * (x - 1.0);"
        "x = (sigma / r_eff_sterics)^6;"

        # Define sigma
        "sigma = (sigma1 + sigma2) / 2;",

        # Define sigma1 and sigma2 (with alchemical scaling)
        "sigma1 = is_unique_old1 * sigma_old1 + is_unique_new1 * sigma_new1 + is_core1 * (max(0.05, lambda * sigma_old1 + lambda * sigma_new1)) + is_environment1 * sigma_old1;",
        "sigma2 = is_unique_old2 * sigma_old2 + is_unique_new2 * sigma_new2 + is_core2 * (max(0.05, lambda * sigma_old2 + lambda * sigma_new2)) + is_environment2 * sigma_old2;",

        # Define epsilon (with rest scaling)
        "epsilon = p1_sterics_rest_scale * p2_sterics_rest_scale * sqrt(epsilon1 * epsilon2);",

        # Define epsilon1 and epsilon2 (with alchemical scaling)
        "epsilon1 = is_unique_old1 * old_epsilon_scaled1 + is_unique_new1 * new_epsilon_scaled1 + is_core1 * (old_epsilon_scaled1 + new_epsilon_scaled1) + is_environment1 * epsilon_old1;",
        "old_epsilon_scaled1 = lambda * epsilon_old1;",
        "new_epsilon_scaled1 = lambda * epsilon_new1;",

        "epsilon2 = is_unique_old2 * old_epsilon_scaled2 + is_unique_new2 * new_epsilon_scaled2 + is_core2 * (old_epsilon_scaled2 + new_epsilon_scaled2) + is_environment2 * epsilon_old2;",
        "old_epsilon_scaled2 = lambda * epsilon_old2;",
        "new_epsilon_scaled2 = lambda * epsilon_new2;",

        # Define rest scale factors (normal rest)
        "p1_sterics_rest_scale = is_rest1 * lambda + is_nonrest_solute1 + is_nonrest_solvent1;",
        "p2_sterics_rest_scale = is_rest2 * lambda + is_nonrest_solute2 + is_nonrest_solvent2;",

        # # Define rest scale factors (scaled water rest)
        # "p1_sterics_rest_scale = lambda_rest_sterics * (is_rest1 + is_nonrest_solvent1 * is_rest2) + 1 * (is_nonrest_solute1 + is_nonrest_solvent1 * is_nonrest_solute2 + is_nonrest_solvent1 * is_nonrest_solvent2);",
        # "p2_sterics_rest_scale = lambda_rest_sterics * (is_rest2 + is_nonrest_solvent2 * is_rest1) + 1 * (is_nonrest_solute2 + is_nonrest_solvent2 * is_nonrest_solute1 + is_nonrest_solvent2 * is_nonrest_solvent1);",

        # Define r_eff
        "r_eff_sterics = sqrt(r^2 + w_sterics^2);",

        # Define 4th dimension terms:
        "w_sterics = w_scale * r_cutoff * (is_unique_old * lambda + is_unique_new * lambda);", # because we want w for unique old atoms to go from 0 to 1 and the opposite for unique new atoms
        "is_unique_old = step(is_unique_old1 + is_unique_old2 - 0.1);", # if at least one of the particles in the interaction is unique old
        "is_unique_new = step(is_unique_new1 + is_unique_new2 - 0.1);", # if at least one of the particles in the interaction is unique new
        "w_scale = 0.3;",
        "r_cutoff = 1.0;"

    ]

_default_sterics_expression = ' '.join(_default_sterics_expression_list)


In [5]:
# Create system and customize it for this experiment
system, positions = get_2_particle_system_and_positions(_default_sterics_expression)
nb_force = system.getForce(0)
nb_force.setUseLongRangeCorrection(False)

# Add global parameters
nb_force.addGlobalParameter(f"lambda", 1.0)

# Add per-particle parameters for rest scaling -- these three sets are disjoint
nb_force.addPerParticleParameter("is_rest")
nb_force.addPerParticleParameter("is_nonrest_solute")
nb_force.addPerParticleParameter("is_nonrest_solvent")

# Add per-particle parameters for alchemical scaling -- these sets are also disjoint
nb_force.addPerParticleParameter('is_environment')
nb_force.addPerParticleParameter('is_core')
nb_force.addPerParticleParameter('is_unique_old')
nb_force.addPerParticleParameter('is_unique_new')

# Add per-particle parameters for defining energy
nb_force.addPerParticleParameter("sigma_old")
nb_force.addPerParticleParameter("sigma_new")
nb_force.addPerParticleParameter("epsilon_old")
nb_force.addPerParticleParameter("epsilon_new")

nb_force.addParticle([1, 0, 0, 0, 0, 1, 0, 0.24, 0.24, 0.37, 0.37])
nb_force.addParticle([1, 0, 0, 0, 1, 0, 0, 0.45, 0.45, 0.15, 0.15])

# Run experiment
run_experiment(system, positions)

0.018584251403808594
0.0002193450927734375
0.00018906593322753906
0.00019788742065429688
0.0001819133758544922
mean:  0.0001970529556274414


# With complex expression and many per particle params, long range correction on

In [4]:
_default_sterics_expression_list = [

        "U_sterics;",

        # Define sterics functional form
        "U_sterics = 4 * epsilon * x * (x - 1.0);"
        "x = (sigma / r_eff_sterics)^6;"

        # Define sigma
        "sigma = (sigma1 + sigma2) / 2;",

        # Define sigma1 and sigma2 (with alchemical scaling)
        "sigma1 = is_unique_old1 * sigma_old1 + is_unique_new1 * sigma_new1 + is_core1 * (max(0.05, lambda * sigma_old1 + lambda * sigma_new1)) + is_environment1 * sigma_old1;",
        "sigma2 = is_unique_old2 * sigma_old2 + is_unique_new2 * sigma_new2 + is_core2 * (max(0.05, lambda * sigma_old2 + lambda * sigma_new2)) + is_environment2 * sigma_old2;",

        # Define epsilon (with rest scaling)
        "epsilon = p1_sterics_rest_scale * p2_sterics_rest_scale * sqrt(epsilon1 * epsilon2);",

        # Define epsilon1 and epsilon2 (with alchemical scaling)
        "epsilon1 = is_unique_old1 * old_epsilon_scaled1 + is_unique_new1 * new_epsilon_scaled1 + is_core1 * (old_epsilon_scaled1 + new_epsilon_scaled1) + is_environment1 * epsilon_old1;",
        "old_epsilon_scaled1 = lambda * epsilon_old1;",
        "new_epsilon_scaled1 = lambda * epsilon_new1;",

        "epsilon2 = is_unique_old2 * old_epsilon_scaled2 + is_unique_new2 * new_epsilon_scaled2 + is_core2 * (old_epsilon_scaled2 + new_epsilon_scaled2) + is_environment2 * epsilon_old2;",
        "old_epsilon_scaled2 = lambda * epsilon_old2;",
        "new_epsilon_scaled2 = lambda * epsilon_new2;",

        # Define rest scale factors (normal rest)
        "p1_sterics_rest_scale = is_rest1 * lambda + is_nonrest_solute1 + is_nonrest_solvent1;",
        "p2_sterics_rest_scale = is_rest2 * lambda + is_nonrest_solute2 + is_nonrest_solvent2;",

        # # Define rest scale factors (scaled water rest)
        # "p1_sterics_rest_scale = lambda_rest_sterics * (is_rest1 + is_nonrest_solvent1 * is_rest2) + 1 * (is_nonrest_solute1 + is_nonrest_solvent1 * is_nonrest_solute2 + is_nonrest_solvent1 * is_nonrest_solvent2);",
        # "p2_sterics_rest_scale = lambda_rest_sterics * (is_rest2 + is_nonrest_solvent2 * is_rest1) + 1 * (is_nonrest_solute2 + is_nonrest_solvent2 * is_nonrest_solute1 + is_nonrest_solvent2 * is_nonrest_solvent1);",

        # Define r_eff
        "r_eff_sterics = sqrt(r^2 + w_sterics^2);",

        # Define 4th dimension terms:
        "w_sterics = w_scale * r_cutoff * (is_unique_old * lambda + is_unique_new * lambda);", # because we want w for unique old atoms to go from 0 to 1 and the opposite for unique new atoms
        "is_unique_old = step(is_unique_old1 + is_unique_old2 - 0.1);", # if at least one of the particles in the interaction is unique old
        "is_unique_new = step(is_unique_new1 + is_unique_new2 - 0.1);", # if at least one of the particles in the interaction is unique new
        "w_scale = 0.3;",
        "r_cutoff = 1.0;"

    ]

_default_sterics_expression = ' '.join(_default_sterics_expression_list)


In [5]:
# Create system and customize it for this experiment
system, positions = get_2_particle_system_and_positions(_default_sterics_expression)
nb_force = system.getForce(0)
nb_force.setUseLongRangeCorrection(True)

# Add global parameters
nb_force.addGlobalParameter(f"lambda", 1.0)

# Add per-particle parameters for rest scaling -- these three sets are disjoint
nb_force.addPerParticleParameter("is_rest")
nb_force.addPerParticleParameter("is_nonrest_solute")
nb_force.addPerParticleParameter("is_nonrest_solvent")

# Add per-particle parameters for alchemical scaling -- these sets are also disjoint
nb_force.addPerParticleParameter('is_environment')
nb_force.addPerParticleParameter('is_core')
nb_force.addPerParticleParameter('is_unique_old')
nb_force.addPerParticleParameter('is_unique_new')

# Add per-particle parameters for defining energy
nb_force.addPerParticleParameter("sigma_old")
nb_force.addPerParticleParameter("sigma_new")
nb_force.addPerParticleParameter("epsilon_old")
nb_force.addPerParticleParameter("epsilon_new")

nb_force.addParticle([1, 0, 0, 0, 0, 1, 0, 0.24, 0.24, 0.37, 0.37])
nb_force.addParticle([1, 0, 0, 0, 1, 0, 0, 0.45, 0.45, 0.15, 0.15])

# Run experiment
run_experiment(system, positions)

0.012380361557006836
0.006872892379760742
0.006464958190917969
0.006218433380126953
0.006335735321044922
mean:  0.0064730048179626465
